In [1]:
!pip install shapely

The directory '/home/jovyan/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/jovyan/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


# Geospatial analysis 


- 14 million trip in NYC


## Goal Compute the utilization 

In [2]:
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.rdd import portable_hash
from pyspark.statcounter import StatCounter


import os
import json

from datetime import datetime
from operator import itemgetter
#from itertools import chain, imap
from shapely.geometry import shape, Point

from matplotlib import pyplot as plt

spark = SparkSession.builder.appName("Taxi")\
        .config("spark.driver.memory", "6g")\
        .config("spark.driver.cores", "8")\
        .getOrCreate()
sc=spark.sparkContext


In [3]:
from  pprint import pprint
def title(s):
    pprint("---- %s -----" %s)    
    
def see(s, v):
    pprint("---- %s -----" %s)
    pprint(v)

# 1. Sample the Data
- Save the sample to a file, because the sampling itself is time-consuming

In [4]:
DATA_DIR = "./geospatial"

In [5]:
taxiRawAll = sc.textFile(DATA_DIR+"/trip_data_1.csv.gz")
header = sc.parallelize(taxiRawAll.take(1))
taxiRaw = taxiRawAll.sample(withReplacement=False, fraction=0.01 ,seed=17)

In [6]:
header.collect()

['medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude']

In [7]:
taxiH = header.union(taxiRaw)
oneFileRdd = taxiH.coalesce(1)
oneFileRdd.saveAsTextFile("trip_data_sample_wheader.csv")

In [8]:
taxiH = sc.parallelize(taxiRawAll.take(14000))
#taxiH = taxiRawAll.mapPartitionsWithIndex(lambda p, i, )


# 2. Read the sample Data
- Save the sample to a file, because the sampling itself is time-consuming

In [9]:
df = spark.read.csv("trip_data_sample_wheader.csv",header=True)
df.toPandas()

,medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
0,2C159C8FCCDE50174CF6CFC07E75F1BA,51C567531541EE45C5EE86E956E46E4C,CMT,1,N,2013-01-07 04:45:14,2013-01-07 04:48:56,1,222,.90,-73.991936,40.749622,-73.978531,40.751308
1,B14118AD4BF381D80035EF546CE7AEFA,9DCEE90343F24D475B3A19A3F10AE801,CMT,1,N,2013-01-10 15:34:30,2013-01-10 16:01:54,3,1643,4.40,-74.010124,40.720692,-73.979279,40.758221
2,20DEC2EBA19D9AD90627DADA0C4FB5C6,F49E5DDD03A4321862B54D5161E4145C,VTS,1,None,2013-01-13 11:07:00,2013-01-13 11:15:00,1,480,.70,-73.9935,40.75592,-73.994934,40.750183
3,4B60321AF8B7EE4ECAE09222711228D7,C95686699914AD2534F6A2EC9FEEE29C,VTS,1,None,2013-01-13 04:36:00,2013-01-13 05:00:00,1,1440,8.61,-73.884262,40.747471,-73.913467,40.838776
4,8B15A77EA92F4291DD8DAB6E7099F92A,CFAF9033E489E987449D999135BC8EF9,VTS,1,None,2013-01-13 10:17:00,2013-01-13 10:32:00,1,900,4.34,-74.000885,40.757973,-74.014748,40.70454
5,F4E581BB51B26B6311826622034AD695,3A929C37D8624B724380E165DF8DEF0E,VTS,1,None,2013-01-13 10:14:00,2013-01-13 10:36:00,6,1320,9.69,-73.984673,40.744621,-73.87384,40.771481
6,4F9B5CF4F0FC8835DF81B36E978A6D52,9AD99D982AD273D72A5A2AC0C3828B1F,VTS,1,None,2013-01-13 11:08:00,2013-01-13 11:16:00,1,480,1.78,-73.956017,40.778728,-73.980064,40.78091
7,A4C090FE7F3048ADB70E51C6947A4CDF,25DB9A08FC899A1052293E8E2159583F,VTS,1,None,2013-01-13 10:30:00,2013-01-13 10:36:00,6,360,1.31,-74.008835,40.714272,-74.016754,40.708076
8,8703B7FD106C380AC6525ED7CA17CF91,10F529F7CA376017ECC3B0B62E64EAA2,VTS,1,None,2013-01-13 10:23:00,2013-01-13 10:30:00,6,420,1.89,-73.979294,40.776741,-73.966705,40.793495
9,463FFB280ACD007D759F0C542D2E3EB7,E643C45208357215A3A5DC8B9E7DCBCE,VTS,1,None,2013-01-13 10:23:00,2013-01-13 10:31:00,1,480,2.89,-74.011841,40.707531,-73.995239,40.739361


# 3. Parse the data handling Time and Geo  types
- We are mainly intersted in each Trip's:
    - Some Unique ID for the car (license)
    - Pick-up location
    - Pick-up time
    - Drop-off location
    - Drop-off time
- We use python datetime class to store dates
- We use shapely to store geo coordinates

In [10]:
def parse(fields):
    license = fields[1]
    pickupTime = datetime.strptime(fields[5], '%Y-%m-%d %H:%M:%S')
    dropoffTime = datetime.strptime(fields[6], '%Y-%m-%d %H:%M:%S')
    try:
        pickupLoc = Point(float(fields[10]), float(fields[11]))
        dropoffLoc = Point(float(fields[12]), float(fields[13]))
    except ValueError:
        pickupLoc = Point(0.0, 0.0)
        dropoffLoc= Point(0.0, 0.0)
    trip = {'pickupTime':pickupTime, 'dropoffTime':dropoffTime, 'pickupLoc':pickupLoc, 'dropoffLoc':dropoffLoc}
    return (license, trip)


taxiParsed = taxiH\
        .map(lambda line: line.split(','))\
        .filter(lambda fields: len(fields) == 14 and fields[0] != "medallion")\
        .map(parse)
taxiParsed.cache()

see("taxiParsed", taxiParsed.take(5))

'---- taxiParsed -----'
[('BA96DE419E711691B9445D6A6307C170',
  {'dropoffLoc': <shapely.geometry.point.Point object at 0x7f0604a7aef0>,
   'dropoffTime': datetime.datetime(2013, 1, 1, 15, 18, 10),
   'pickupLoc': <shapely.geometry.point.Point object at 0x7f0604a7ae80>,
   'pickupTime': datetime.datetime(2013, 1, 1, 15, 11, 48)}),
 ('9FD8F69F0804BDB5549F40E9DA1BE472',
  {'dropoffLoc': <shapely.geometry.point.Point object at 0x7f0604a7af60>,
   'dropoffTime': datetime.datetime(2013, 1, 6, 0, 22, 54),
   'pickupLoc': <shapely.geometry.point.Point object at 0x7f0604a7ae48>,
   'pickupTime': datetime.datetime(2013, 1, 6, 0, 18, 35)}),
 ('9FD8F69F0804BDB5549F40E9DA1BE472',
  {'dropoffLoc': <shapely.geometry.point.Point object at 0x7f0603db5080>,
   'dropoffTime': datetime.datetime(2013, 1, 5, 18, 54, 23),
   'pickupLoc': <shapely.geometry.point.Point object at 0x7f0604a7afd0>,
   'pickupTime': datetime.datetime(2013, 1, 5, 18, 49, 41)}),
 ('51EE87E3205C985EF8431D850C786310',
  {'dropoffLoc':

# 4. Process Time: 
## 4.1 Explore the distribution of trip duration (hours)

In [11]:
def hours(trip):
    d= trip['dropoffTime'] - trip['pickupTime']
    return int( ((d.days)*24) + (d.seconds/3600))
    #return d
    
hoursCount = taxiParsed.values().map(hours).countByValue().items()
sortedHoursCount = sorted(hoursCount, key=itemgetter(0), reverse=False)

for val in sortedHoursCount:
    print(val)    
taxiParsed.count()

(0, 13997)
(1, 2)


13999

## 4.2 Remove trips with -ve durations and longer than 3 hours

In [12]:
def goodHour(hrs):
    return 0 <= hrs and hrs < 3

taxiClean = taxiParsed.filter(lambda x: goodHour( hours(x[1]) )).cache()
taxiParsed.unpersist()

see("taxiClean", taxiClean.take(5))
taxiClean.count()

'---- taxiClean -----'
[('BA96DE419E711691B9445D6A6307C170',
  {'dropoffLoc': <shapely.geometry.point.Point object at 0x7f0603d18c18>,
   'dropoffTime': datetime.datetime(2013, 1, 1, 15, 18, 10),
   'pickupLoc': <shapely.geometry.point.Point object at 0x7f0603d18c50>,
   'pickupTime': datetime.datetime(2013, 1, 1, 15, 11, 48)}),
 ('9FD8F69F0804BDB5549F40E9DA1BE472',
  {'dropoffLoc': <shapely.geometry.point.Point object at 0x7f0603d18b70>,
   'dropoffTime': datetime.datetime(2013, 1, 6, 0, 22, 54),
   'pickupLoc': <shapely.geometry.point.Point object at 0x7f0603d18f28>,
   'pickupTime': datetime.datetime(2013, 1, 6, 0, 18, 35)}),
 ('9FD8F69F0804BDB5549F40E9DA1BE472',
  {'dropoffLoc': <shapely.geometry.point.Point object at 0x7f0603d18c88>,
   'dropoffTime': datetime.datetime(2013, 1, 5, 18, 54, 23),
   'pickupLoc': <shapely.geometry.point.Point object at 0x7f0603d18d30>,
   'pickupTime': datetime.datetime(2013, 1, 5, 18, 49, 41)}),
 ('51EE87E3205C985EF8431D850C786310',
  {'dropoffLoc': 

13999

# 5.Process Geo data
## 5.1 Geet a list of NY Zones (boroughs)

In [13]:
with open(DATA_DIR+'/nyc-boroughs.geojson', 'r') as f:
        geo = json.load(f)
features = geo['features']
for f in features:
    f["shape"] = shape(f['geometry'])

see("features", features[:3])

'---- features -----'
[{'geometry': {'coordinates': [[[-74.05050806403247, 40.566422034160816],
                                [-74.04998352562575, 40.56639592492827],
                                [-74.04931640362088, 40.56588774778044],
                                [-74.04923629842045, 40.5653627363681],
                                [-74.05002620158643, 40.565318180621134],
                                [-74.05090601705089, 40.5660943421306],
                                [-74.05067916748614, 40.5663108457364],
                                [-74.05107159803778, 40.5667224933978],
                                [-74.05050806403247, 40.566422034160816]]],
               'type': 'Polygon'},
  'id': 0,
  'properties': {'@id': 'http://nyc.pediacities.com/Resource/Borough/Staten_Island',
                 'borough': 'Staten Island',
                 'boroughCode': 5},
  'shape': <shapely.geometry.polygon.Polygon object at 0x7f0600c8b7b8>,
  'type': 'Feature'},
 {'geometry': 

## 5.2 Sort the zones by area and broadcast to executors

In [14]:
areaSortedFeatures = sorted(features, key=lambda f: (int(f['properties']["boroughCode"]), -f["shape"].area), reverse=False)
areaSortedFeatures

[{'geometry': {'coordinates': [[[-73.92640556921117, 40.87762147653734],
     [-73.92629548795446, 40.87748846858917],
     [-73.92645784900961, 40.877258189008735],
     [-73.92619292267656, 40.877064419414374],
     [-73.92555292248268, 40.87710258898463],
     [-73.92508215701339, 40.87739761061557],
     [-73.92457036990588, 40.877428912671306],
     [-73.92329476733927, 40.87726443545978],
     [-73.92239768169318, 40.876780223141196],
     [-73.92244468226002, 40.87569183576377],
     [-73.9226680400197, 40.874981991369474],
     [-73.92261383040437, 40.87440727210196],
     [-73.92217677288461, 40.87367506500938],
     [-73.92125541079864, 40.87308244882503],
     [-73.92055602106613, 40.8729987739631],
     [-73.92013405705723, 40.873192834478274],
     [-73.91979301672357, 40.8736065126604],
     [-73.92062699165689, 40.87423481112875],
     [-73.92080255630358, 40.87419405886667],
     [-73.92070010598987, 40.87439193360941],
     [-73.92118186508978, 40.87513784085064],
    

In [15]:
bFeatures = sc.broadcast(areaSortedFeatures)
see("areaSortedFeatures", areaSortedFeatures[:3])

'---- areaSortedFeatures -----'
[{'geometry': {'coordinates': [[[-73.92640556921117, 40.87762147653734],
                                [-73.92629548795446, 40.87748846858917],
                                [-73.92645784900961, 40.877258189008735],
                                [-73.92619292267656, 40.877064419414374],
                                [-73.92555292248268, 40.87710258898463],
                                [-73.92508215701339, 40.87739761061557],
                                [-73.92457036990588, 40.877428912671306],
                                [-73.92329476733927, 40.87726443545978],
                                [-73.92239768169318, 40.876780223141196],
                                [-73.92244468226002, 40.87569183576377],
                                [-73.9226680400197, 40.874981991369474],
                                [-73.92261383040437, 40.87440727210196],
                                [-73.92217677288461, 40.87367506500938],
               

## 5.3 Convert long/lat to  zone

In [16]:
def borough(trip):
    for f in bFeatures.value:
        #contains checks if the 
        if f['shape'].contains(trip["dropoffLoc"]):
            return str(f['properties']["borough"])
    return None

boroughCount = taxiClean.values().map(borough).countByValue().items()

see("boroughCount", list(boroughCount))

'---- boroughCount -----'
[('Manhattan', 11288),
 ('Queens', 1161),
 ('Brooklyn', 1014),
 (None, 410),
 ('Bronx', 119),
 ('Staten Island', 7)]


## 5.4 Remove trips that started/ended in unknown zones

In [17]:
def hasZero(trip):
    zero = Point(0.0, 0.0)
    return (zero == trip["pickupLoc"] or zero == trip["dropoffLoc"])

taxiDone = taxiClean.filter(lambda x: not hasZero(x[1])).cache()

In [18]:
boroughCount = taxiDone.values().map(borough).countByValue().items()
see("boroughCount", list(boroughCount))

'---- boroughCount -----'
[('Manhattan', 11286),
 ('Queens', 1161),
 ('Brooklyn', 1014),
 (None, 84),
 ('Bronx', 119),
 ('Staten Island', 7)]


## 6 Sessionize trips
- Look at sorted list of trips for each driver
- Once the difference between any 2 consecurtive trips is longer than 4 hours, we consider that a new session has started

In [19]:
def borough_new(trip):
    for f in bFeatures.value:
        #contains checks if the 
        if f['shape'].contains(trip):
            return str(f['properties']["borough"])
    return None

dtaRdd=taxiDone.map(lambda x:(x[0],borough_new(x[1]['dropoffLoc']),x[1]['dropoffTime'],borough_new(x[1]['pickupLoc']),x[1]['pickupTime']))
dtaDF = spark.createDataFrame(dtaRdd, ['lic','dropoffLoc','dropoffTime','pickupLoc','pickupTime'])


In [20]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import col,when
from pyspark.sql.types import TimestampType

def duration_fun(t2,t1,l1,l2,r):
    if l1!=l2:
        return 1
    else:
        try:
            total = datetime.strptime(str(t1), "%Y-%m-%d %H:%M:%S") - datetime.strptime(str(t2), "%Y-%m-%d %H:%M:%S")
            to = int(total.days)*24 + int(total.seconds)/3600
            if to <= 4:
                return 0
            else:
                return 1
        except:
            return -1

sorteddf = dtaDF.sort('lic','pickupTime')
my_window = Window.partitionBy().orderBy("lic")
newdf = sorteddf.withColumn("prev_value",F.lag(sorteddf.dropoffTime).over(my_window))\
.withColumn("prev_lic",F.lag(sorteddf.lic).over(my_window))
newdf.createOrReplaceTempView("ti")
spark.udf.register("duration_fun", duration_fun)
sqlQuery = "select ROW_NUMBER() OVER (ORDER BY lic) as ID,lic,dropoffLoc,dropoffTime,pickupLoc,pickupTime,duration_fun(pickupTime,prev_value,lic,prev_lic,ROW_NUMBER() OVER (ORDER BY lic)) as check FROM ti"
d= spark.sql(sqlQuery)
f = d.withColumn("Session",when(col("check")==1,col("ID")).otherwise(F.lag(d.ID).over(my_window)))\
.select("Session","lic","dropoffLoc","dropoffTime","pickupLoc")
f.show(10)


+-------+--------------------+----------+-------------------+---------+
|Session|                 lic|dropoffLoc|        dropoffTime|pickupLoc|
+-------+--------------------+----------+-------------------+---------+
|      1|001C8AAB90AEE49F3...|  Brooklyn|2013-01-13 04:41:00|Manhattan|
|      1|001C8AAB90AEE49F3...| Manhattan|2013-01-13 06:30:00|Manhattan|
|      2|001C8AAB90AEE49F3...| Manhattan|2013-01-13 06:44:00|Manhattan|
|      4|0025133AD810DBE80...| Manhattan|2013-01-13 02:46:00|Manhattan|
|      5|002C093A2CB9FD40C...| Manhattan|2013-01-13 12:08:00|Manhattan|
|      6|00374328FBA75FBFC...|    Queens|2013-01-13 04:03:00|Manhattan|
|      7|0046F1E91AA13DEDE...| Manhattan|2013-01-13 11:22:00|   Queens|
|      8|00567B1CBFD51DDFA...| Manhattan|2013-01-13 09:37:00|Manhattan|
|      8|00567B1CBFD51DDFA...| Manhattan|2013-01-13 09:50:00|Manhattan|
|      9|00567B1CBFD51DDFA...| Manhattan|2013-01-13 10:41:00|Manhattan|
+-------+--------------------+----------+-------------------+---

In [21]:
f1 = f.select('lic').groupby('lic').count()
f1.show(5)

+--------------------+-----+
|                 lic|count|
+--------------------+-----+
|02856AFC22881ABCA...|    2|
|03A2D28F831C5C3E5...|    3|
|0FBF11956EE14B253...|    1|
|138B0A7B7D3B898E4...|    1|
|13CD9D132F9DFE9BD...|    2|
+--------------------+-----+
only showing top 5 rows



In [22]:
f1.filter(('count > 2')).show(20)

+--------------------+-----+
|                 lic|count|
+--------------------+-----+
|03A2D28F831C5C3E5...|    3|
|428AE5AF18511D16B...|    4|
|42D2B75CA34A867A4...|    3|
|49EE1E8ECFA1C6D35...|    3|
|669FA40A7222D4DC2...|    4|
|7DA853581E8C09723...|    5|
|83279FE1F7D548407...|    3|
|8821CA9250E34A869...|    4|
|980A15AE7BF88E611...|    9|
|A7EE9AEDB7325F55F...|    3|
|D4F2AE0988ECB2E42...|    4|
|E7E6F9F97F18C35E0...|    5|
|F41C758E0A9022F59...|    4|
|FA36E516BD1822D19...|    5|
|015D33FBAB8A7C5CE...|    5|
|0B6E99C5380BF98FA...|    4|
|0F51E0C88BD265002...|    3|
|12D07C9D59FADFFDB...|    3|
|19269BF987DE73F57...|    4|
|59FC2FA712CDAE3E4...|    3|
+--------------------+-----+
only showing top 20 rows

